In [1]:
# toggle to save space
mode = '_mean' if False else '_all'
print(mode)

_all


In [2]:
# TODO: add https://www.kaggle.com/corochann/optuna-tutorial-for-hyperparameter-optimization

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from pandas.core.dtypes.dtypes import CategoricalDtype
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from datetime import date 
import holidays


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [4]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [5]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [6]:
building = loadFile('building_metadata')
pre_train = loadFile('train')
#test = loadFile('test')

In [7]:
def merge(x):
    weather_processed_df = pd.read_pickle(f'../input/ashrae-energy-prediction-pickles/weather_processed{mode}.pickle')
    weather_processed_df = weather_processed_df.dropna(axis=1)
    x = x.merge(building, on=['building_id'], how='left')
    gc.collect()
    x = x.merge(weather_processed_df, on=['site_id', 'timestamp'], how='left')
    gc.collect()
    x['site_id'] = x['site_id'].astype('int8');
    x['cloud_coverage'] = x['cloud_coverage'].astype('float16')
    gc.collect()
    return x
        
train = merge(pre_train) 
gc.collect()
print('!!!! Warning we are missing weather for '+ str(train['air_temperature'].isnull().sum())+' rows')

print(train.dtypes)

!!!! Warning we are missing weather for 0 rows
building_id                             int16
meter                                    int8
timestamp                      datetime64[ns]
meter_reading                         float32
site_id                                  int8
primary_use                            object
square_feet                           float32
year_built                            float16
floor_count                           float16
air_temperature                       float16
dew_temperature                       float16
cloud_coverage                        float16
precip_depth_1_hr                     float16
wind_direction                        float16
wind_speed                            float16
sea_level_pressure                    float16
relative_humidity                     float16
air_temperature_rmean_3               float16
air_temperature_rmax_3                float16
air_temperature_rmin_3                float16
cloud_coverage_rmean_3           

In [8]:
# See holiday notebook to generate, this is optional
holiday_df = None
if path.exists('../input/ashrae-energy-prediction-pickles/holiday_df.pickle'):
    holiday_df = pd.read_pickle('../input/ashrae-energy-prediction-pickles/holiday_df.pickle')
if holiday_df is not None:
    print(holiday_df.sample(20))

          site_id           timestamp                                 holiday
34936069        4 2018-11-22 14:00:00                            Thanksgiving
54850645       14 2017-12-25 23:00:00                           Christmas Day
44870862        9 2018-11-12 21:00:00                 Veterans Day (Observed)
35243765        5 2017-04-14 21:00:00                             Good Friday
19841210        4 2016-12-25 09:00:00                           Christmas Day
2492968         4 2016-02-15 17:00:00                   Washington's Birthday
4436116        11 2016-03-25 10:00:00                             Good Friday
46332704       11 2018-12-26 20:00:00                              Boxing Day
33538400        4 2017-02-20 12:00:00                   Washington's Birthday
38352270        7 2018-02-19 04:00:00                              Family Day
37096844        6 2017-10-09 12:00:00                            Columbus Day
8133997         2 2016-05-30 16:00:00                           

In [9]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        # TODO: try week of year as numerical 
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('uint8') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('uint8')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [10]:
class MeterReadingLog1p(TransformerMixin):
  
    def transform(self, df, **transform_params):
        if 'meter_reading' in df.columns:
            df['meter_reading_log1p'] = np.log1p(df['meter_reading'])
            df = df.drop('meter_reading', axis=1)
        return df
    
    def fit(self, X, y=None, **fit_params):
        return self
print(train.sample(20, random_state=42))
print(MeterReadingLog1p().transform(train.sample(20, random_state=42)))
gc.collect()

          building_id  meter           timestamp  meter_reading  site_id  \
14245562         1324      1 2016-09-16 16:00:00       0.000000       14   
1282718          1013      0 2016-01-24 06:00:00      32.000099       10   
13883790          229      1 2016-09-10 07:00:00     567.655029        2   
4781820           217      3 2016-04-01 01:00:00       0.000000        2   
10415393         1434      0 2016-07-10 04:00:00      65.750000       15   
1057008          1047      0 2016-01-20 04:00:00      90.983299       12   
4507399           911      1 2016-03-26 20:00:00     295.063995        9   
19478829         1039      0 2016-12-18 23:00:00      16.900000       12   
8955615           265      0 2016-06-14 06:00:00     128.369995        2   
13799839          896      0 2016-09-08 19:00:00     300.000000        9   
15647011          973      0 2016-10-11 11:00:00     247.000000        9   
2524294           813      0 2016-02-16 08:00:00      10.958300        8   
10016102    

[20 rows x 65 columns]
          building_id  meter           timestamp  site_id  \
14245562         1324      1 2016-09-16 16:00:00       14   
1282718          1013      0 2016-01-24 06:00:00       10   
13883790          229      1 2016-09-10 07:00:00        2   
4781820           217      3 2016-04-01 01:00:00        2   
10415393         1434      0 2016-07-10 04:00:00       15   
1057008          1047      0 2016-01-20 04:00:00       12   
4507399           911      1 2016-03-26 20:00:00        9   
19478829         1039      0 2016-12-18 23:00:00       12   
8955615           265      0 2016-06-14 06:00:00        2   
13799839          896      0 2016-09-08 19:00:00        9   
15647011          973      0 2016-10-11 11:00:00        9   
2524294           813      0 2016-02-16 08:00:00        8   
10016102          870      0 2016-07-03 02:00:00        8   
3915750           898      0 2016-03-15 03:00:00        9   
17217526          903      0 2016-11-08 09:00:00        9   
1

[20 rows x 65 columns]


14

In [11]:
class CreateMeterDescDF(TransformerMixin):

    def transform(self, df, **transform_params):
        global _building_meter_desc_DF
        if 'meter_reading_log1p' in df.columns:
            cols = ['mean']
            if mode == '_all':
                cols = ['mean','max','min','std']
            group = df.groupby(['building_id','meter'])['meter_reading_log1p']
            desc_DF = group.describe(percentiles=[.05, .25, .5, .75, .95 ])
            desc_DF = desc_DF.reset_index()
            col_dict = {}
            for col in desc_DF.columns:
                if col not in ['building_id', 'meter', 'count']:
                    col_dict[col] = 'meter_' + col
            _building_meter_desc_DF = desc_DF.rename(columns=col_dict).drop('count', axis=1)
            gc.collect()
        return df 
    def fit(self, X, y=None, **fit_params):
        return self

#if 'meter_mean' not in train.columns:
#    print(building_meter_desc_DF)
#    train = train.merge(building_meter_desc_DF, on=['building_id','meter'], how='left')
#    #test = test.merge(building_meter_desc_DF, on=['building_id','meter'], how='left')
#    del building_meter_desc_DF
CreateMeterDescDF().transform(
    AddTimeFeatures().transform(
        MeterReadingLog1p().transform(
            train.sample(2000, random_state=0)
        )
    )
)    
print(_building_meter_desc_DF)
gc.collect()

      building_id  meter  meter_mean  meter_std  meter_min  meter_5%  \
0               1      0    4.921724        NaN   4.921724  4.921724   
1               2      0    0.000000        NaN   0.000000  0.000000   
2               3      0    5.727568   0.128644   5.636602  5.645699   
3               5      0    0.000000        NaN   0.000000  0.000000   
4               6      0    0.000000        NaN   0.000000  0.000000   
5               8      0    5.930674        NaN   5.930674  5.930674   
6               9      1    5.300433   2.169901   3.766081  3.919516   
7              11      0    6.159797        NaN   6.159797  6.159797   
8              12      0    5.636602        NaN   5.636602  5.636602   
9              14      0    0.000000        NaN   0.000000  0.000000   
10             14      1    8.555042        NaN   8.555042  8.555042   
11             15      0    5.232920        NaN   5.232920  5.232920   
12             15      1    7.265242   0.810718   6.452293  6.53

[1349 rows x 11 columns]


21

In [12]:
class MergeMeterDescDF(TransformerMixin):
  
    def transform(self, df, **transform_params):
        # drop any columns to add
        dropCols =  [x for x in _building_meter_desc_DF.columns if x not in ['building_id', 'meter']]
        df = df.drop(dropCols, axis=1, errors='ignore') 
        return df.merge(_building_meter_desc_DF, on=['building_id','meter'], how='left')

    def fit(self, X, y=None, **fit_params):
        return self

print(MergeMeterDescDF().transform(train.sample(2000, random_state=0)))

      building_id  meter           timestamp  meter_reading  site_id  \
0             774      1 2016-08-07 08:00:00      36.128899        6   
1             206      0 2016-10-04 14:00:00     226.270004        2   
2            1269      0 2016-11-29 10:00:00      28.670799       14   
3             951      0 2016-10-10 04:00:00     113.000000        9   
4             656      0 2016-05-01 21:00:00      32.700001        5   
5              36      0 2016-06-05 19:00:00     178.830994        0   
6            1262      0 2016-07-19 05:00:00      73.739998       14   
7              52      0 2016-03-17 14:00:00       0.000000        0   
8            1133      2 2016-07-10 17:00:00     984.375000       13   
9            1123      0 2016-10-02 09:00:00      18.243999       13   
10           1237      0 2016-10-06 18:00:00      85.000000       14   
11            960      1 2016-11-17 10:00:00      90.066498        9   
12             11      0 2016-09-25 21:00:00     472.332001     

[2000 rows x 74 columns]


In [13]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [14]:
# following this thread
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113254#latest-663021
class RmBuilt2017(TransformerMixin):
# TO  
    def transform(self, df, **transform_params):
        # 954 leading zeros
        df = df.query('not (building_id == 954 & timestamp <= "2016-08-12")')
        df = df.query('not (building_id == 954 & timestamp <= "2016-10-12" & timestamp >= "2016-10-18")')
        
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [15]:
# remove big offenders in the error analysis reports
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113254#latest-663021
class RmEAOffeners(TransformerMixin):
# TO  
    def transform(self, df, **transform_params):
        # 1072 new steam install
        df = df.query('not (building_id == 1072 & timestamp <= "2016-07-27")')
        
        # 783
        df = df.query('not (building_id == 783 & timestamp <= "2016-12-10")')
        
        # 1264
        # not sure what to do with that one
        
        # 1021
        # drop zeros for hot water
        
        # 799
        df = df.query('not (building_id == 799 & meter == 0 & timestamp <= "2016-09-12")')
        
        # 693
        df = df.query('not (building_id == 693 & meter == 0 & timestamp <= "2016-07-11")')
        
        return df

    def fit(self, X, y=None, **fit_params):
        return self
    


In [16]:
# TODO: write filter to remove any 0 meter reading that continue more than N days (try 3)
# Also we need to account for this by meter

In [17]:
# TODO: try rolling with power

In [18]:
    
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['timestamp','site_id'], how='left')
            df['holiday'] = df['holiday'].astype('category')
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
if holiday_df is not None:
    print(holiday_df.columns)
    print(AddHolidays().transform(train.head(20))[['holiday','timestamp']])

Index(['site_id', 'timestamp', 'holiday'], dtype='object')
           holiday  timestamp
0   New Year's Day 2016-01-01
1   New Year's Day 2016-01-01
2   New Year's Day 2016-01-01
3   New Year's Day 2016-01-01
4   New Year's Day 2016-01-01
5   New Year's Day 2016-01-01
6   New Year's Day 2016-01-01
7   New Year's Day 2016-01-01
8   New Year's Day 2016-01-01
9   New Year's Day 2016-01-01
10  New Year's Day 2016-01-01
11  New Year's Day 2016-01-01
12  New Year's Day 2016-01-01
13  New Year's Day 2016-01-01
14  New Year's Day 2016-01-01
15  New Year's Day 2016-01-01
16  New Year's Day 2016-01-01
17  New Year's Day 2016-01-01
18  New Year's Day 2016-01-01
19  New Year's Day 2016-01-01


In [19]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['timestamp','site_id'], how='left')
            df = df.drop(df[df['holiday'].notnull()].index)
            df = df.drop(['holiday'], axis=1)
            gc.collect()
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
#print(train.head(100000).merge(building, on='building_id', how='left').columns)
print(RmHolidays().transform(train.head(100000)))

       building_id  meter           timestamp  meter_reading  site_id  \
55121            0      0 2016-01-02 00:00:00       0.000000        0   
55122            1      0 2016-01-02 00:00:00       0.000000        0   
55123            2      0 2016-01-02 00:00:00       0.000000        0   
55124            3      0 2016-01-02 00:00:00       0.000000        0   
55125            4      0 2016-01-02 00:00:00       0.000000        0   
55126            5      0 2016-01-02 00:00:00       0.000000        0   
55127            6      0 2016-01-02 00:00:00       0.000000        0   
55128            7      0 2016-01-02 00:00:00       0.000000        0   
55129            8      0 2016-01-02 00:00:00       0.000000        0   
55130            9      0 2016-01-02 00:00:00       0.000000        0   
55131           10      0 2016-01-02 00:00:00       0.000000        0   
55132           11      0 2016-01-02 00:00:00       0.000000        0   
55133           12      0 2016-01-02 00:00:00      

[41839 rows x 65 columns]


In [20]:
class SetCatTypes(TransformerMixin):
    
    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col]= df[col].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [21]:
class GetDummies(TransformerMixin):
    
    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        df = pd.get_dummies(df, columns=self._cols )
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [22]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [23]:
# TODO: Play with scaling cloud coverage

In [24]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [25]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        tmp_df = train.drop_duplicates(['site_id','building_id'])[['site_id','building_id','year_built']]
        year_built_median = tmp_df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in tmp_df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del tmp_df, year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(ImputeYearBuilt().transform(train.sample(20))['building_age'])

4880893      70
17552803     52
19411126     91
9413053      62
17828758     70
17043335     53
9052558      70
12436223     70
13149334     70
920559       30
4518331      70
20096597     70
17172102     67
16774262     70
11747614     70
18592925     70
15484918     23
5243600      85
8195674      76
9295477     107
Name: building_age, dtype: uint8


In [26]:
class ImputeFloorCount(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        tmp_df = train.drop_duplicates(['site_id','building_id'])[['site_id','building_id','floor_count']]
        floors_median = tmp_df['floor_count'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in tmp_df.groupby(['site_id'])['floor_count'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['floor_count'].isnull()) & (df['site_id'] == i), 'floor_count'] = i_median
            else:
                df.loc[(df['floor_count'].isnull()) & (df['site_id'] == i), 'floor_count'] = floors_median
        del tmp_df, floors_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

print(ImputeFloorCount().transform(train.sample(20))['floor_count'])

4362271     3.0
9337212     3.0
15452690    3.0
7197042     3.0
5493815     3.0
9514756     3.0
10593353    3.0
1117716     3.0
11074910    3.0
1938049     3.0
9848394     3.0
1026143     3.0
13618016    3.0
14617725    3.0
18403293    3.0
6034679     3.0
1392606     3.0
11009154    3.0
13232615    3.0
8988778     7.0
Name: floor_count, dtype: float16


In [27]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [29]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [30]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [31]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [32]:
class MergeWeatherDescHDOW(TransformerMixin):
    def transform(self, df, **transform_params):
        desc_h_dow_df = pd.read_csv('weather_desc_h_dow.csv', index_col=0, dtype={'building_id': np.int16, 'meter': np.int8, 'meter_h_d_mean':np.float32,
                                                     'meter_h_d_std':np.float32, 'meter_h_d_min':np.float32,
                                                     'meter_h_d_25%':np.float32, 'meter_h_d_50%':np.float32,
                                                     'meter_h_d_75%':np.float32, 'meter_h_d_max':np.float32 })
        mrgCols = ['site_id', 'dayofweek','hour']
        dropCols = list(x for x in desc_h_dow_df.columns if x not in mrgCols)
        df.drop(mrgCols, errors='ignore')
        df = df.merge(desc_h_dow_df, on=mrgCols, how='left')
        
        del desc_h_dow_df
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

print(MergeWeatherDescHDOW().transform(AddTimeFeatures().transform(ConvertToDatetime().transform(train.sample(20)))))

    building_id  meter           timestamp  meter_reading  site_id  \
0           801      2 2016-09-27 17:00:00      30.090000        7   
1           709      0 2016-06-26 23:00:00       6.300000        5   
2            83      0 2016-08-11 14:00:00       7.951800        0   
3           892      0 2016-03-15 23:00:00     149.000000        9   
4          1295      1 2016-12-10 07:00:00     183.733002       14   
5          1269      1 2016-12-09 13:00:00       1.341000       14   
6           545      0 2016-01-02 23:00:00      59.619999        3   
7          1370      0 2016-08-03 08:00:00      16.200001       15   
8           920      1 2016-08-09 11:00:00     410.522003        9   
9          1348      2 2016-05-14 19:00:00    1999.540039       15   
10          203      1 2016-01-21 21:00:00     463.731995        2   
11           61      0 2016-01-17 13:00:00       0.000000        0   
12          532      0 2016-11-27 21:00:00     113.949997        3   
13          216     

[20 rows x 102 columns]


In [33]:
class MergeDescHDOW(TransformerMixin):
    def transform(self, df, **transform_params):
        desc_h_dow_df = pd.read_csv('desc_h_dow.csv', index_col=0, dtype={'building_id': np.int16, 'meter': np.int8, 'meter_h_d_mean':np.float32,
                                                     'meter_h_d_std':np.float32, 'meter_h_d_min':np.float32,
                                                     'meter_h_d_25%':np.float32, 'meter_h_d_50%':np.float32,
                                                     'meter_h_d_75%':np.float32, 'meter_h_d_max':np.float32 })
        desc_h_dow_df = desc_h_dow_df.dropna(axis=1)
        mrgCols = ['building_id', 'meter', 'dayofweek','hour']
        dropCols = list(x for x in desc_h_dow_df.columns if x not in mrgCols)
        df.drop(mrgCols, errors='ignore')
        df = df.merge(desc_h_dow_df, on=mrgCols, how='left')
        
        del desc_h_dow_df
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

print(MergeDescHDOW().transform(AddTimeFeatures().transform(ConvertToDatetime().transform(train.sample(20)))))

    building_id  meter           timestamp  meter_reading  site_id  \
0          1140      0 2016-07-07 12:00:00     599.145020       13   
1          1291      0 2016-03-05 22:00:00     487.903015       14   
2           951      2 2016-02-13 08:00:00     158.600006        9   
3          1193      1 2016-03-03 19:00:00      80.967598       13   
4          1242      0 2016-09-25 03:00:00       9.000000       14   
5           118      0 2016-04-12 01:00:00     232.000000        1   
6           443      0 2016-09-18 11:00:00      82.250000        3   
7           650      0 2016-05-01 08:00:00     123.500000        4   
8          1332      0 2016-01-22 15:00:00      76.199997       15   
9          1052      0 2016-04-17 13:00:00      26.017500       12   
10         1009      3 2016-05-25 18:00:00       8.357900       10   
11           16      0 2016-01-22 11:00:00       0.000000        0   
12          191      0 2016-07-02 03:00:00      45.029999        2   
13         1101     

In [34]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [35]:
pd.options.display.max_seq_items = 2000

In [36]:
%%time
x_pre_pipes = Pipeline(
    steps=[
        ('meterReadingLog1p',MeterReadingLog1p()),
        ('rmS0M0', RmS0M0()),
        ('rmBuilt2017', RmBuilt2017()),
        ('RmEAOffeners', RmEAOffeners()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('logSquareFeet', LogSquareFeet()),
        #('rmHolidays', RmHolidays()),
        #('addHolidays', AddHolidays()),
        #('createMeterDescDF', CreateMeterDescDF()), # note declares a globe variable to pass
        #('mergeMeterDescDF', MergeMeterDescDF()), # populates both test and train from global
        ('mergeDescHDOW', MergeDescHDOW()),
        #('mergeWeatherDescHDOW', MergeWeatherDescHDOW()),
        ('GC', GC())
    ]
)

train = x_pre_pipes.transform(merge(pre_train))
print(train.columns)

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'dew_temperature', 'cloud_coverage', 'precip_depth_1_hr',
       'wind_direction', 'wind_speed', 'sea_level_pressure',
       'relative_humidity', 'air_temperature_rmean_3',
       'air_temperature_rmax_3', 'air_temperature_rmin_3',
       'cloud_coverage_rmean_3', 'cloud_coverage_rmax_3',
       'cloud_coverage_rmin_3', 'dew_temperature_rmean_3',
       'dew_temperature_rmax_3', 'dew_temperature_rmin_3',
       'precip_depth_1_hr_rmean_3', 'precip_depth_1_hr_rmax_3',
       'precip_depth_1_hr_rmin_3', 'sea_level_pressure_rmean_3',
       'sea_level_pressure_rmax_3', 'sea_level_pressure_rmin_3',
       'wind_direction_rmean_3', 'wind_direction_rmax_3',
       'wind_direction_rmin_3', 'wind_speed_rmean_3', 'wind_speed_rmax_3',
       'wind_speed_rmin_3', 'relative_humidity_rmean_3',
       'relative_humidity_rmax_3', 'relative_humidity_rmin_3

In [37]:
 train.isna().any()

building_id                    False
meter                          False
timestamp                      False
site_id                        False
primary_use                    False
square_feet                    False
year_built                      True
floor_count                     True
air_temperature                False
dew_temperature                False
cloud_coverage                 False
precip_depth_1_hr              False
wind_direction                 False
wind_speed                     False
sea_level_pressure             False
relative_humidity              False
air_temperature_rmean_3        False
air_temperature_rmax_3         False
air_temperature_rmin_3         False
cloud_coverage_rmean_3         False
cloud_coverage_rmax_3          False
cloud_coverage_rmin_3          False
dew_temperature_rmean_3        False
dew_temperature_rmax_3         False
dew_temperature_rmin_3         False
precip_depth_1_hr_rmean_3      False
precip_depth_1_hr_rmax_3       False
p

In [38]:
# pre_a_pipes is for preprocessing that doesn't change impute
# values
cat_cols = ['site_id', 'meter', 'primary_use']

x_fold_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('imputeFloorCount', ImputeFloorCount()),
        ('dropCols', DropCols(['timestamp','square_feet', 'year_built'])),
        ('getDummies', GetDummies(cat_cols)),
        ('GC', GC())
    ]
)

sample_train_X = x_fold_pipes.transform(train.sample(20))
print(sample_train_X.columns)
print(sample_train_X.shape)
print(sample_train_X.dtypes)
nans = sample_train_X.isna().any()
print(nans[nans==True].index.tolist())

Index(['building_id', 'floor_count', 'air_temperature', 'dew_temperature',
       'cloud_coverage', 'precip_depth_1_hr', 'wind_direction', 'wind_speed',
       'sea_level_pressure', 'relative_humidity', 'air_temperature_rmean_3',
       'air_temperature_rmax_3', 'air_temperature_rmin_3',
       'cloud_coverage_rmean_3', 'cloud_coverage_rmax_3',
       'cloud_coverage_rmin_3', 'dew_temperature_rmean_3',
       'dew_temperature_rmax_3', 'dew_temperature_rmin_3',
       'precip_depth_1_hr_rmean_3', 'precip_depth_1_hr_rmax_3',
       'precip_depth_1_hr_rmin_3', 'sea_level_pressure_rmean_3',
       'sea_level_pressure_rmax_3', 'sea_level_pressure_rmin_3',
       'wind_direction_rmean_3', 'wind_direction_rmax_3',
       'wind_direction_rmin_3', 'wind_speed_rmean_3', 'wind_speed_rmax_3',
       'wind_speed_rmin_3', 'relative_humidity_rmean_3',
       'relative_humidity_rmax_3', 'relative_humidity_rmin_3',
       'air_temperature_rmean_72', 'air_temperature_rmax_72',
       'air_temperature_rm

In [39]:
# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
# trying no shuffle https://www.kaggle.com/c/ashrae-energy-prediction/discussion/115851#latest-666115
folds = 2
kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=42)


In [100]:

def cvFitScore(train, n_estimators):
    models_p = []

    for train_index, val_index in kf.split(train, train['building_id']):
        print('starting fold')
        f_train = x_fold_pipes.transform(train.loc[train_index].drop('building_id', axis=1))
        f_train_x = f_train.drop('meter_reading_log1p', axis=1)
        
        feature_cols = list(x for x in f_train_x.columns if x not in cat_cols)
        scaler_x = MinMaxScaler()
        scaler_y = MinMaxScaler()
        f_train_x[feature_cols]= scaler_x.fit_transform(f_train_x[feature_cols])
        
        f_train_y = scaler_y.fit_transform(f_train['meter_reading_log1p'].values.reshape(-1, 1))
        #print(f_train_y)
        
        f_val = x_fold_pipes.transform(train.loc[train_index].drop('building_id', axis=1))
        f_val_x = f_val.drop('meter_reading_log1p', axis=1)
        f_val_x[feature_cols] = scaler_x.transform(f_val_x[feature_cols])
        f_val_y = scaler_y.transform(f_val['meter_reading_log1p'].values.reshape(-1, 1))
        
        lin_reg = LinearRegression()
        lin_reg.fit(f_train_x, f_train_y)
        
        score = rmsle(
            scaler_y.inverse_transform(f_val_y), 
            scaler_y.inverse_transform(lin_reg.predict(f_val_x)).clip(0)
        )
        
        models_p.append({'model':lin_reg, 'score':score, 'scaler_x':scaler_x, 'scaler_y':scaler_y})        
        del f_train, f_val
        gc.collect()
    return models_p

In [101]:
%%time
model_ps = cvFitScore(train, n_estimators= 10)
print(np.sum(mp['score'] for mp in  model_ps)/len(model_ps))

starting fold
[[0.35438445]
 [0.3536773 ]
 [0.35285667]
 ...
 [0.        ]
 [0.30052522]
 [0.07976945]]
starting fold
[[0.18876964]
 [0.01882374]
 [0.        ]
 ...
 [0.22758782]
 [0.22744872]
 [0.22692734]]
0.3736899658357409
Wall time: 16min 54s


In [42]:
print(model_ps)

[{'model': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False), 'score': 0.38305382561172324, 'scaler_x': MinMaxScaler(copy=True, feature_range=(0, 1)), 'scaler_y': MinMaxScaler(copy=True, feature_range=(0, 1))}, {'model': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False), 'score': 0.36432586156551927, 'scaler_x': MinMaxScaler(copy=True, feature_range=(0, 1)), 'scaler_y': MinMaxScaler(copy=True, feature_range=(0, 1))}]


In [43]:
raise("error")

TypeError: exceptions must derive from BaseException

In [ ]:
# %%time
# ## Single fit single model

# gbm = LGBMRegressor(**gbm_params)
# f_train_X, f_train_y = getInFoldXY(train.index)
# gbm.fit(f_train_X, f_train_y)

In [102]:
#del train
#gc.collect()
test = loadFile('test')
print(test.shape)
test = merge(test)
print(test.shape)
test = x_pre_pipes.transform(test)
print(test.shape)
test = x_fold_pipes.transform(test)
print(test.shape)
test = test.drop('row_id', axis=1)



#print(test.sample(n=20,  random_state=42))
print(test.shape)
#print(test.dtypes)

(41697600, 4)
(41697600, 65)
(41697600, 75)
(41697600, 106)
(41697600, 105)


In [103]:
print(test[test.isna().any(axis=1)])



Empty DataFrame
Columns: [building_id, floor_count, air_temperature, dew_temperature, cloud_coverage, precip_depth_1_hr, wind_direction, wind_speed, sea_level_pressure, relative_humidity, air_temperature_rmean_3, air_temperature_rmax_3, air_temperature_rmin_3, cloud_coverage_rmean_3, cloud_coverage_rmax_3, cloud_coverage_rmin_3, dew_temperature_rmean_3, dew_temperature_rmax_3, dew_temperature_rmin_3, precip_depth_1_hr_rmean_3, precip_depth_1_hr_rmax_3, precip_depth_1_hr_rmin_3, sea_level_pressure_rmean_3, sea_level_pressure_rmax_3, sea_level_pressure_rmin_3, wind_direction_rmean_3, wind_direction_rmax_3, wind_direction_rmin_3, wind_speed_rmean_3, wind_speed_rmax_3, wind_speed_rmin_3, relative_humidity_rmean_3, relative_humidity_rmax_3, relative_humidity_rmin_3, air_temperature_rmean_72, air_temperature_rmax_72, air_temperature_rmin_72, cloud_coverage_rmean_72, cloud_coverage_rmax_72, cloud_coverage_rmin_72, dew_temperature_rmean_72, dew_temperature_rmax_72, dew_temperature_rmin_72, pre

In [104]:
l1 =  x_fold_pipes.transform(train).columns.tolist()
l2 =  test.columns.tolist()
print(list(x for x in l1 if x not in l2))
print(list(x for x in l2 if x not in l1))
nans = test.isna().any()
print(nans[nans==True].index.tolist())



['meter_reading_log1p']
[]
[]


In [109]:
def mpkPredict(mpk, test_X):
    pred = mpk['model'].predict(mpk['scaler_x'].transform(test_X.drop('building_id', axis=1)))
    #print(pred)
    print( mpk['scaler_y'].inverse_transform(pred))
    return mpk['scaler_y'].inverse_transform(pred)

In [110]:
def predMeters(test_X):
    #print(np.sum([mpkPredict(mpk, test_X) for mpk in model_ps], axis=0)/2)
    preds = np.expm1(
        np.sum([mpkPredict(mpk, test_X) for mpk in model_ps], axis=0)/folds)
    #test_y['meter_reading_log1p'] = preds
    return preds.tolist()
    
print(predMeters(test.sample(n=20,  random_state=42)))    


[[4.67291706]
 [3.10923964]
 [5.20347983]
 [1.0015581 ]
 [6.23485388]
 [3.11471933]
 [5.85591731]
 [8.27204084]
 [5.35410679]
 [5.66812517]
 [3.86073055]
 [6.73969434]
 [5.88196195]
 [4.86154728]
 [4.27112697]
 [4.29272339]
 [2.65597271]
 [4.29917008]
 [4.15698829]
 [5.02912904]]
[[4.70938709]
 [2.68742619]
 [5.58965691]
 [1.09268871]
 [6.77564358]
 [2.51217819]
 [5.70390984]
 [7.96072757]
 [5.79346815]
 [5.32833235]
 [3.83766046]
 [6.8733582 ]
 [6.33101141]
 [4.40718423]
 [4.11542661]
 [4.36398476]
 [2.79935795]
 [4.79362606]
 [4.10788282]
 [5.09379791]]
[[107.97865948409759], [17.143872748936445], [219.64793335562703], [1.8494426160310016], [667.6419585740026], [15.667300681153645], [322.7312107930801], [3347.889929857622], [262.42994478745175], [243.2589074446931], [45.95527265582099], [902.7260499160241], [447.75930718468635], [101.96259349058296], [65.23948863891218], [74.81939074098273], [14.297131575969276], [93.29216242468682], [61.32954516443162], [156.82131476175957]]


In [111]:
# Predict using cross val models ensemble 
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    res.append(predMeters(test.iloc[i:i+step_size]))
    i+=step_size
    gc.collect()



  0%|                                                                                          | 0/834 [00:00<?, ?it/s]

[[5.26598051]
 [4.56187279]
 [2.25595546]
 ...
 [5.33989184]
 [6.06614394]
 [5.28867287]]
[[5.50764239]
 [4.83934013]
 [2.50885763]
 ...
 [5.41421548]
 [6.13932182]
 [5.35631529]]



  0%|                                                                                  | 1/834 [00:00<04:14,  3.27it/s]

[[4.72233096]
 [4.89516678]
 [4.92488603]
 ...
 [4.691548  ]
 [7.52696439]
 [5.20667094]]
[[4.7843414 ]
 [4.97216238]
 [4.98251091]
 ...
 [4.76248376]
 [7.60955456]
 [5.28655137]]



  0%|▏                                                                                 | 2/834 [00:00<03:59,  3.48it/s]

...


100%|███████████████████████████████████████████████████████████████████████████████▊| 832/834 [16:11<00:04,  2.09s/it]

[[3.05289555]
 [5.57110245]
 [2.93350281]
 ...
 [6.04941478]
 [0.09541104]
 [2.66425671]]
[[3.66763508]
 [5.3868773 ]
 [3.58068808]
 ...
 [6.58830617]
 [0.10651571]
 [2.2301888 ]]



100%|███████████████████████████████████████████████████████████████████████████████▉| 833/834 [16:13<00:02,  2.11s/it]

[[6.09025457]
 [2.39768599]
 [0.55518913]
 ...
 [1.90538434]
 [5.17824103]
 [1.6669212 ]]
[[6.61029277]
 [2.4498291 ]
 [0.87256484]
 ...
 [1.92531015]
 [5.146411  ]
 [1.67678424]]



100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [16:15<00:00,  2.11s/it]

In [112]:
# Save using cross val models ensemble 
res = np.concatenate(res)
print(len(res))
submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_meter.csv', index=False)
submission.shape

41697600


(41697600, 2)

In [ ]:
# # Predict single model fit
# i=0
# res=[]
# step_size = 50000
# for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
#    #res.append(np.expm1(sum([model.predict(test_X.iloc[i:i+step_size]) for model in models])/folds))
#    res.append(np.expm1(gbm.predict(test_X.iloc[i:i+step_size])))
#    i+=step_size
    